In [1]:
# Import necessary libraries
import pandas as pd
import tensorflow_decision_forests as tfdf
import tensorflow as tf



2024-02-28 07:01:14.014344: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 07:01:14.014400: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 07:01:14.016853: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-28 07:01:14.027584: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-28 07:01:14.716212: W tensorflow/compiler/tf2

In [2]:

# Load your dataset
data_path = 'combined_dataset_updated.csv'  # Update this path
data = pd.read_csv(data_path)



In [3]:
# Split the data into features and target
X = data.iloc[:, :-1]
y = data.iloc[:, -1]



In [4]:
# Convert to TensorFlow dataset and split into train, validation, and test sets
def df_to_tf_dataset(features, target):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), target))
    return dataset



In [7]:
# Splitting the data (adjust split sizes as needed)
train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.15)
test_size = len(data) - train_size - val_size

full_dataset = df_to_tf_dataset(X, y)
full_dataset = full_dataset.shuffle(buffer_size=len(data))

# Adjusted dataset preparation without shuffle operation
train_dataset = full_dataset.take(train_size).batch(16)
val_dataset = full_dataset.skip(train_size).take(val_size).batch(16) # Ensure validation dataset is correctly defined
test_dataset = full_dataset.skip(train_size + val_size).batch(16)


In [9]:
# Train a Random Forest model with dataset check disabled (if needed)
model = tfdf.keras.RandomForestModel(check_dataset=False)
model.fit(train_dataset)

Use /tmp/tmpc0isgm7y as temporary training directory
Reading training dataset...
Training dataset read in 0:00:13.821231. Found 55853 examples.
Training model...


[INFO 24-02-28 07:13:05.5599 +0530 kernel.cc:1233] Loading model from path /tmp/tmpc0isgm7y/model/ with prefix 88ef81fa3e484008


Model trained in 0:00:02.289315
Compiling model...


[INFO 24-02-28 07:13:05.8403 +0530 decision_forest.cc:660] Model loaded with 300 root(s), 198508 node(s), and 10 input feature(s).
[INFO 24-02-28 07:13:05.8404 +0530 abstract_model.cc:1344] Engine "RandomForestOptPred" built
[INFO 24-02-28 07:13:05.8404 +0530 kernel.cc:1061] Use fast generic engine


Model compiled.


In [10]:
# Evaluate the model on the test set
evaluation = model.evaluate(test_dataset)



749/749 [==============================] - 4s 4ms/step - loss: 0.0000e+00


In [11]:
# Plotting feature importances
model.summary()



Model: "random_forest_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (10):
	max_pedestrian_iou
	max_pedestrian_position_x
	max_pedestrian_position_y
	max_pedestrian_velocity_x
	max_pedestrian_velocity_y
	max_vehicle_iou
	max_vehicle_position_x
	max_vehicle_position_y
	max_vehicle_velocity_x
	max_vehicle_velocity_y

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1. "max_pedestrian_position_y"  0.478980 ################
    2.    "max_vehicle_position_y"  0.245697 ######
    3. "max_pedestrian_position_x"  0.228046 #####
    4.    "max_vehicle_position_x"  0.149804 ##
    5. "max_pedestrian_velocity_x"  0.126010 #
    6.    "max_vehicle_velocity_x"  0.

In [12]:
# Built-in TensorFlow Decision Forests Plotting
# Note: This requires TensorFlow Decision Forests version >= 0.2.0

# Plotting model structure and feature importances
tfdf.model_plotter.plot_model_in_colab(model, tree_idx=0, max_depth=3)
tfdf.model_plotter.plot_feature_importances(model)


AttributeError: module 'tensorflow_decision_forests.component.model_plotter.model_plotter' has no attribute 'plot_feature_importances'

In [13]:

# Note: The `plot_model_in_colab` function is designed to work in Google Colab and might not render directly in Jupyter.
# For Jupyter, you can use `model.make_inspector().export_to_tensorboard('/path/to/logdir')` and visualize with TensorBoard.

# Advanced model insights
inspector = model.make_inspector()
print("Model type:", inspector.model_type())
print("Number of trees:", inspector.num_trees())
print("Objective:", inspector.objective())
print("Input features:", inspector.features())
print("Evaluation:", inspector.evaluation())



Model type: RANDOM_FOREST
Number of trees: 300
Objective: Classification(label=__LABEL, class=None, num_classes=2)
Input features: ["max_pedestrian_iou" (1; #1), "max_pedestrian_position_x" (1; #2), "max_pedestrian_position_y" (1; #3), "max_pedestrian_velocity_x" (1; #4), "max_pedestrian_velocity_y" (1; #5), "max_vehicle_iou" (1; #6), "max_vehicle_position_x" (1; #7), "max_vehicle_position_y" (1; #8), "max_vehicle_velocity_x" (1; #9), "max_vehicle_velocity_y" (1; #10)]
Evaluation: Evaluation(num_examples=55853, accuracy=0.9917819991764095, loss=0.029005823127058388, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)


In [ ]:
# Optional: Save your model for later use
model.save('your_model_path')